In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

spark = SparkSession.builder \
    .master("yarn") \
    .appName("SparkMLOct10-RF500-yarn") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 05:08:32 INFO SparkEnv: Registering MapOutputTracker
23/11/16 05:08:32 INFO SparkEnv: Registering BlockManagerMaster
23/11/16 05:08:32 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/11/16 05:08:32 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
bucket_path = "gs://fifa_players"

In [5]:
df_15 = spark.read.csv(bucket_path + "/data/players_15.csv",header=True, inferSchema= True)

In [6]:
df_16 = (spark.read.csv(bucket_path + "/data/players_16.csv",header=True, inferSchema= True))
df_17 = (spark.read.csv(bucket_path + "/data/players_17.csv",header=True, inferSchema= True))
df_18 = (spark.read.csv(bucket_path + "/data/players_18.csv",header=True, inferSchema= True))
df_19 = (spark.read.csv(bucket_path + "/data/players_19.csv",header=True, inferSchema= True))
df_20 = (spark.read.csv(bucket_path + "/data/players_20.csv",header=True, inferSchema= True))
df_21 = (spark.read.csv(bucket_path + "/data/players_21.csv",header=True, inferSchema= True))
df_22 = (spark.read.csv(bucket_path + "/data/players_22.csv",header=True, inferSchema= True))

In [7]:
df_15 = df_15.withColumn('Year', lit(2015))
df_16 = df_16.withColumn('Year', lit(2016))
df_17 = df_17.withColumn('Year', lit(2017))
df_18 = df_18.withColumn('Year', lit(2018))
df_19 = df_19.withColumn('Year', lit(2019))
df_20 = df_20.withColumn('Year', lit(2020))
df_21 = df_21.withColumn('Year', lit(2021))
df_22 = df_22.withColumn('Year', lit(2022))

In [8]:
complete_df = df_15.union(df_16).union(df_17).union(df_18).union(df_19).union(df_20).union(df_21).union(df_22)

complete_df.show(1,vertical = True)

23/11/16 05:09:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24 00:00:00  
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [9]:
complete_df = complete_df.withColumn('UID', monotonically_increasing_id())
df_read = complete_df
df_read = df_read.sample(False, 0.1)

In [10]:

# Analytical Queries Section
from pyspark.sql import functions as F

club_num = input("Enter the number of clubs you want to analyze: ")
print("You entered: " + club_num)


df_2022 = df_read.filter((df_read['Year']== 2022) & (df_read["club_contract_valid_until"] == 2023))

club_counts = df_2022.groupBy("club_name").count().orderBy(F.desc("count"))
club_counts.show(int(club_num))

Enter the number of clubs you want to analyze:  10
You entered: 10


+--------------------+-----+
|           club_name|count|
+--------------------+-----+
| Stade Malherbe Caen|    5|
|          SC Cambuur|    5|
|            Al Hazem|    4|
|     Strømsgodset IF|    4|
|TSV Egger Glas Ha...|    3|
|Central Coast Mar...|    3|
|          AC Ajaccio|    3|
|    APOEL Nicosia FC|    3|
|FC Paços de Ferreira|    3|
|       Bradford City|    3|
+--------------------+-----+
only showing top 10 rows



In [11]:
# Check if the club_name is null and age is greater than 27
older_players = df_read.filter((df_read["age"] > 27) & (df_read["club_name"].isNotNull()))

# Group by club_name and Year
club_year_counts = older_players.groupBy("club_name", "Year").agg(F.count("UID").alias("total_count"))

# Group by club_name to calculate average
club_avg_counts = club_year_counts.groupBy("club_name").agg(F.avg("total_count").alias("average_count"))

# Order by average in descending order
sorted_clubs = club_avg_counts.orderBy(F.desc("average_count"))

# Display the top Y clubs, handling tie scenarios
num_club = int(input("Enter the number of top clubs you want to see: "))
print("You entered: " + str(num_club))
threshold = sorted_clubs.limit(num_club).select("average_count").collect()[num_club-1]["average_count"]
    
filtered_result = sorted_clubs.filter(sorted_clubs["average_count"] >= threshold).show()

# Result
filtered_result


Enter the number of top clubs you want to see:  10
You entered: 10


+--------------------+-------------+
|           club_name|average_count|
+--------------------+-------------+
|Club Deportivo Jo...|          4.0|
|  Sandecja Nowy Sącz|          4.0|
|   Crucero del Norte|          4.0|
|          VfB Lübeck|          4.0|
|               Hajer|          3.5|
|        Gyeongnam FC|          3.5|
|  Santiago Wanderers|          3.0|
|  SpVgg Unterhaching|          3.0|
|Club Deportivo El...|          3.0|
|     Rapid București|          3.0|
|Demir Grup Sivasspor|          3.0|
|       Nueva Chicago|          3.0|
|         FC Botoşani|          3.0|
|              Paraná|          3.0|
|         CD Feirense|          3.0|
|      Ventforet Kofu|          3.0|
|         KV Kortrijk|          3.0|
|Évian Thonon Gail...|          3.0|
|        Nashville SC|          3.0|
|Club Universitari...|          3.0|
+--------------------+-------------+
only showing top 20 rows



In [12]:
# Group by Year and nation_position
df_read_filter = df_read.filter(df_read["nation_position"].isNotNull())
nation_position_counts = df_read_filter.groupBy("Year", "nation_position").agg(F.count("UID").alias("total_count"))

# Find the max count for each year
max_count = nation_position_counts.groupBy("Year").agg(F.max("total_count").alias("max_count"))
max_count = max_count.withColumnRenamed("Year", "Year2")

# Join the two table to find which position has the max count for each year
top_nation_positions = nation_position_counts.join(max_count, (nation_position_counts["Year"] == max_count["Year2"]) & 
                                                   (nation_position_counts["total_count"] == max_count["max_count"]))

# Result
top_nation_positions.select("Year", "nation_position", "max_count").orderBy(F.desc("Year")).show()

+----+---------------+---------+
|Year|nation_position|max_count|
+----+---------------+---------+
|2022|            SUB|       30|
|2021|            SUB|       62|
|2020|            SUB|       53|
|2019|            SUB|       49|
|2018|            SUB|       56|
|2017|            SUB|       51|
|2016|            SUB|       38|
|2015|            SUB|       57|
+----+---------------+---------+

